# 10_03: Visualizing COVID-19 data

In [ ]:
import math
import collections
import dataclasses
import datetime

import numpy as np
import pandas as pd
import plotly.express as px

# pretty-printing code for pandas Series and DataFrames
pd.Series.nice = lambda series, format='{:,.0f}': pd.DataFrame(series).style.format(format)
pd.DataFrame.nice = lambda frame, format='{:,.0f}': frame.style.format(format)

In [ ]:
covid19 = pd.read_csv('covid19.csv.gz', parse_dates=['date'], dtype_backend='pyarrow')

In [ ]:
final = covid19.groupby('country').last()

final['cases_per_million'] = final.total_cases / (final.population / 1.0e6)
final['deaths_per_million'] = final.total_deaths / (final.population / 1.0e6)
final['excess_per_million'] = final.total_excess / (final.population / 1.0e6)

In [ ]:
final.nlargest(10, 'excess_per_million')[['cases_per_million', 'deaths_per_million', 'excess_per_million']].nice()

In [ ]:
fig = px.scatter(final, x='population', y='excess_per_million', color='continent',
                 title='Excess Deaths per Million by Population')
fig.show()

In [ ]:
fig = px.scatter(final.reset_index().dropna(subset='excess_per_million'),
                 x='population', y='excess_per_million', color='continent',
                 hover_name='country', title='Excess Deaths per Million by Population')
fig.show()

In [ ]:
fig = px.scatter(final.reset_index().dropna(subset='excess_per_million'),
                 x='gdp_per_capita', y='excess_per_million', color='continent',
                 hover_name='country', title='Excess Deaths per Million by Population')
fig.show()

In [ ]:
fig = px.scatter(final.reset_index().dropna(subset='excess_per_million'),
                 x='percent_fully_vaccinated', y='excess_per_million', color='continent',
                 hover_name='country', title='Excess Deaths per Million by Population')
fig.show()

In [ ]:
fig = px.line(covid19, x='date', y='total_cases', color='country')
fig.show()

In [ ]:
fig = px.line(covid19, x='date', y='total_cases', color='country')

fig.update_xaxes(range=['2020-01-01', '2022-12-31'])
fig.show()

In [ ]:
covid19.groupby('country').total_cases >= 5

In [ ]:
covid19[covid19.total_cases >= 5].groupby('country').head(1)

In [ ]:
t0 = covid19[covid19.total_cases >= 5].groupby('country').head(1).set_index('country').date

In [ ]:
covid19.set_index('country').date - t0

In [ ]:
covid19['t'] = (covid19.set_index('country').date - t0).dt.days.values # take values to avoid merge

In [ ]:
fig = px.line(covid19, x='t', y='total_cases', color='country')

fig.update_xaxes(range=[0, 2*365])
fig.show()

In [ ]:
fig = px.line(covid19, x='t', y='total_cases', color='country', log_y=True)

fig.update_xaxes(range=[0, 2*365])
fig.update_layout(width=700)
fig.show()

In [ ]:
focused = covid19[(covid19.population > 5e7) & (covid19.date < '2020-06-30')]

In [ ]:
fig = px.line(focused, x='t', y='total_cases', color='country', log_y=True)

fig.update_xaxes(range=[0, 175])
fig.update_yaxes(range=[1,7])
fig.update_layout(width=800, height=400)
fig.show()

In [ ]:
fig = px.line(focused, x='t', y='total_cases', color='continent', log_y=True,
              hover_name='country', template='simple_white', color_discrete_sequence=px.colors.qualitative.Set3)

fig.update_xaxes(range=[0, 175], title='Days since 5th case')
fig.update_yaxes(range=[1, 7], title='Total cases')
fig.update_layout(width=600, height=400)

fig.show()

In [ ]:
fig = px.line(focused, x='t', y='total_cases', color='continent', log_y=True,
              hover_name='country', template='simple_white', color_discrete_sequence=px.colors.qualitative.Set3)

fig.update_xaxes(range=[0, 175], title='Days since 5th case')
fig.update_yaxes(range=[1, 7], title='Total cases')
fig.update_layout(width=600, height=400)

for tdouble in [2, 5, 10]:
    fig.add_scatter(x=np.array([0, 150]), y=np.array([5, 5 * 2**(150 / tdouble)]),
                    mode='lines', showlegend=False, line=dict(color='black', dash='dot'))

fig.update_layout(annotations=[{'x': 20,  'y': np.log10(1e6), 'text': '2 days',  'showarrow': False},
                               {'x': 70,  'y': np.log10(1e6), 'text': '5 days',  'showarrow': False},
                               {'x': 150, 'y': np.log10(4e4), 'text': '10 days', 'showarrow': False}])

fig.show()

In [ ]:
fig.write_html('cases.html')